In [3]:
import tensorflow as tf
import numpy as np
import nltk
import sacrebleu
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional, Dense, Embedding, Input
from tensorflow.keras.models import Model


In [39]:
# Example small dataset
english_sentences = [
    "hello", "how are you", "what is your name", "where are you from", "good morning","I am learning machine learning","Can you help me?","This is a pen.","Where are you from?","I love programming."
]

hindi_sentences = [
    "नमस्ते", "आप कैसे हैं", "आपका नाम क्या है", "आप कहाँ से हैं", "सुप्रभात","मैं मशीन लर्निंग सीख रहा हूँ","क्या आप मेरी मदद कर सकते हैं?","यह एक कलम है","आप कहाँ से हैं" , "मुझे प्रोग्रामिंग बहुत पसंद है"
] 
# Tokenization
def tokenize(sentences):
    tokenizer = Tokenizer(filters='', lower=False)
    tokenizer.fit_on_texts(sentences)
    return tokenizer, tokenizer.texts_to_sequences(sentences)

eng_tokenizer, eng_sequences = tokenize(english_sentences)
hin_tokenizer, hin_sequences = tokenize(hindi_sentences)

# Padding sequences
max_len = max(max(len(seq) for seq in eng_sequences), max(len(seq) for seq in hin_sequences))
eng_sequences = pad_sequences(eng_sequences, maxlen=max_len, padding='post')
hin_sequences = pad_sequences(hin_sequences, maxlen=max_len, padding='post')

# Vocabulary size
eng_vocab_size = len(eng_tokenizer.word_index) + 1
hin_vocab_size = len(hin_tokenizer.word_index) + 1


In [40]:
def build_rnn_nmt():
    # Encoder
    enc_inputs = Input(shape=(max_len,))
    enc_emb = Embedding(eng_vocab_size, 64)(enc_inputs)
    enc_output, enc_state = SimpleRNN(64, return_state=True)(enc_emb)

    # Decoder
    dec_inputs = Input(shape=(max_len,))
    dec_emb = Embedding(hin_vocab_size, 64)(dec_inputs)
    dec_rnn = SimpleRNN(64, return_sequences=True)(dec_emb, initial_state=[enc_state])
    dec_output = Dense(hin_vocab_size, activation='softmax')(dec_rnn)

    model = Model([enc_inputs, dec_inputs], dec_output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

rnn_model = build_rnn_nmt()
rnn_model.summary()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_23      │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_22        │ (None, 7, 64)     │      1,728 │ input_layer_22[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_23        │ (None, 7, 64)     │      1,920 │ input_layer_23[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_6        │ [(None, 64),      │      8,256 │ embedding_22[0][… │
│ (SimpleRNN)         │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn_7        │ (None, 7, 64)     │      8,256 │ embedding_23[0][… │
│ (SimpleRNN)         │                   │            │ simple_rnn_6[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 7, 30)     │      1,950 │ simple_rnn_7[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 22,110 (86.37 KB)

 Trainable params: 22,110 (86.37 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
def build_lstm_nmt():
    enc_inputs = Input(shape=(max_len,))
    enc_emb = Embedding(eng_vocab_size, 64)(enc_inputs)
    enc_output, state_h, state_c = LSTM(64, return_state=True)(enc_emb)

    dec_inputs = Input(shape=(max_len,))
    dec_emb = Embedding(hin_vocab_size, 64)(dec_inputs)
    dec_lstm = LSTM(64, return_sequences=True)(dec_emb, initial_state=[state_h, state_c])
    dec_output = Dense(hin_vocab_size, activation='softmax')(dec_lstm)

    model = Model([enc_inputs, dec_inputs], dec_output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

lstm_model = build_lstm_nmt()
lstm_model.summary()


Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24      │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_25      │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_24        │ (None, 7, 64)     │      1,728 │ input_layer_24[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_25        │ (None, 7, 64)     │      1,920 │ input_layer_25[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_16 (LSTM)      │ [(None, 64),      │     33,024 │ embedding_24[0][… │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_17 (LSTM)      │ (None, 7, 64)     │     33,024 │ embedding_25[0][… │
│                     │                   │            │ lstm_16[0][1],    │
│                     │                   │            │ lstm_16[0][2]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 7, 30)     │      1,950 │ lstm_17[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 71,646 (279.87 KB)

 Trainable params: 71,646 (279.87 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
def build_bilstm_nmt(vocab_size_eng, vocab_size_hin, embedding_dim, hidden_units, max_seq_length):
    # Encoder
    encoder_input = Input(shape=(max_seq_length,))
    encoder_embedding = Embedding(input_dim=vocab_size_eng, output_dim=embedding_dim)(encoder_input)
    
    # Bidirectional LSTM for encoding the input sequence
    encoder_bilstm = Bidirectional(LSTM(hidden_units, return_state=True))(encoder_embedding)
    encoder_output, forward_h, forward_c, backward_h, backward_c = encoder_bilstm
    
    # Concatenate the forward and backward hidden states
    state_h = tf.concat([forward_h, backward_h], axis=-1)  # (2 * hidden_units,)
    state_c = tf.concat([forward_c, backward_c], axis=-1)  # (2 * hidden_units,)
    
    # Decoder
    decoder_input = Input(shape=(max_seq_length,))
    decoder_embedding = Embedding(input_dim=vocab_size_hin, output_dim=embedding_dim)(decoder_input)
    
    # Decoder LSTM
    decoder_lstm = LSTM(2 * hidden_units, return_sequences=True, return_state=True)
    decoder_output, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
    
    # Output layer (softmax)
    decoder_dense = Dense(vocab_size_hin, activation='softmax')
    decoder_output = decoder_dense(decoder_output)
    
    # Build and compile the model
    model = Model(inputs=[encoder_input, decoder_input], outputs=decoder_output)
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [43]:

bilstm_model = build_lstm_nmt()
bilstm_model.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_26      │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_27      │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_26        │ (None, 7, 64)     │      1,728 │ input_layer_26[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_27        │ (None, 7, 64)     │      1,920 │ input_layer_27[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_18 (LSTM)      │ [(None, 64),      │     33,024 │ embedding_26[0][… │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_19 (LSTM)      │ (None, 7, 64)     │     33,024 │ embedding_27[0][… │
│                     │                   │            │ lstm_18[0][1],    │
│                     │                   │            │ lstm_18[0][2]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 7, 30)     │      1,950 │ lstm_19[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 71,646 (279.87 KB)

 Trainable params: 71,646 (279.87 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
epochs = 10
batch_size = 16


In [45]:
rnn_history = rnn_model.fit([eng_sequences, hin_sequences], hin_sequences, epochs=epochs, batch_size=batch_size)
lstm_history = lstm_model.fit([eng_sequences, hin_sequences], hin_sequences, epochs=epochs, batch_size=batch_size)

Epoch 1/10


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 3.4261
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.1571 - loss: 3.3103
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.4000 - loss: 3.1930
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5000 - loss: 3.0680
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.5286 - loss: 2.9338
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5143 - loss: 2.7931
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5143 - loss: 2.6512
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5143 - loss: 2.5147
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.5143 - loss: 2.3892
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.5000 - loss: 2.2784
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 3.4085
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1429 - loss: 3.3908
Epoch 3/10

In [46]:

bilstm_history = bilstm_history = bilstm_model.fit([eng_sequences, hin_sequences], hin_sequences, epochs=epochs, batch_size=batch_size)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0143 - loss: 3.4000
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.4143 - loss: 3.3830
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.4429 - loss: 3.3652
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4429 - loss: 3.3460
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.4429 - loss: 3.3247
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.4429 - loss: 3.3006
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4429 - loss: 3.2729
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.4429 - loss: 3.2408
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4429 - loss: 3.2030
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.4429 - loss: 3.1584


In [47]:
def evaluate_bleu(model):
    predictions = model.predict([eng_sequences, hin_sequences])
    predicted_sentences = []
    
    for pred in predictions:
        pred_tokens = np.argmax(pred, axis=-1)
        predicted_sentence = " ".join([hin_tokenizer.index_word.get(idx, "") for idx in pred_tokens])
        predicted_sentences.append(predicted_sentence)

    reference_sentences = [" ".join(sent) for sent in hindi_sentences]
    bleu_score = sacrebleu.corpus_bleu(predicted_sentences, [reference_sentences]).score
    return bleu_score

print("RNN BLEU Score:", evaluate_bleu(rnn_model))
print("LSTM BLEU Score:", evaluate_bleu(lstm_model))
print("Bi-LSTM BLEU Score:", evaluate_bleu(bilstm_model))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
RNN BLEU Score: 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
LSTM BLEU Score: 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
Bi-LSTM BLEU Score: 0.0


In [48]:
import math
import numpy as np

def calculate_perplexity(history):
    """Calculates perplexity using the stored training history."""
    if 'loss' not in history.history:
        raise ValueError("Loss key not found in history.")

    losses = history.history['loss']
    
    if not losses:
        raise ValueError("Loss history is empty. Ensure the model was trained properly.")

    avg_loss = sum(losses) / len(losses)

    try:
        perplexity = np.exp(avg_loss)  # Using numpy to prevent overflow
    except OverflowError:
        perplexity = float('inf')  # Assign infinity if overflow occurs

    return perplexity

print("RNN Perplexity:", calculate_perplexity(rnn_history))
print("LSTM Perplexity:", calculate_perplexity(lstm_history))
print("Bi-LSTM Perplexity:", calculate_perplexity(bilstm_history))


RNN Perplexity: 17.38797640948555
LSTM Perplexity: 27.52658299469946
Bi-LSTM Perplexity: 27.098592069375428
